In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

%matplotlib inline

#plt.style.use('fivethirtyeight')

In [3]:
data_train = pd.read_csv('../kag_exp_data/train.csv')
data_train.head()

NameError: name 'pd' is not defined

In [3]:
data_train = data_train[data_train.is_booking == 1]

In [4]:
data_train.shape

(3000693, 24)

In [5]:
print data_train.columns

Index([u'date_time', u'site_name', u'posa_continent', u'user_location_country',
       u'user_location_region', u'user_location_city',
       u'orig_destination_distance', u'user_id', u'is_mobile', u'is_package',
       u'channel', u'srch_ci', u'srch_co', u'srch_adults_cnt',
       u'srch_children_cnt', u'srch_rm_cnt', u'srch_destination_id',
       u'srch_destination_type_id', u'is_booking', u'cnt', u'hotel_continent',
       u'hotel_country', u'hotel_market', u'hotel_cluster'],
      dtype='object')


In [6]:
data_train.srch_destination_id.unique().shape

(36933,)

In [7]:
destinations = pd.read_csv('../kag_exp_data/destinations.csv')
destinations.tail()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
62101,64989,-2.145876,-2.158016,-2.216621,-2.158016,-2.006209,-2.102505,-2.216621,-2.157203,-2.216621,...,-2.216621,-2.158016,-2.158016,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.152024
62102,64990,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.176802,-2.203479,-2.203479,-2.203479,...,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.008885
62103,64991,-2.190567,-2.208308,-2.208308,-2.208308,-2.208308,-2.135493,-2.208308,-2.208308,-1.909808,...,-2.188935,-2.208308,-2.175594,-2.208308,-2.208308,-2.199347,-2.208308,-2.208308,-2.208308,-2.208308
62104,64992,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,...,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686
62105,64993,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,...,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457,-2.179457


In [8]:
destinations.srch_destination_id.unique().shape

(62106,)

Create a subset of the training dataset to work with and test out certain methods.

Can use if the dataset is too large, but after dropping all rows with 0 in 'is_booking' (just clicks, not reservation), the training dataset has been narrowed down to 3 million. This is a managable size without having to wait too long to process.

In [9]:
data_train.reset_index(drop=True, inplace=True)
data_train

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
1,2014-02-27 18:01:32,2,3,66,318,52078,NaN,756,0,1,...,0,1,8291,1,1,1,2,50,191,2
2,2013-06-15 15:38:05,30,4,195,548,56440,NaN,1048,0,1,...,0,1,1385,1,1,1,0,185,185,58
3,2014-11-23 18:02:20,30,4,195,991,47725,NaN,1048,0,0,...,0,1,8803,1,1,1,3,151,69,36
4,2014-01-03 16:30:17,2,3,66,462,41898,2454.8588,1482,0,1,...,0,2,12009,1,1,1,2,50,680,95
5,2014-01-03 16:44:56,2,3,66,462,41898,2454.8588,1482,0,1,...,0,1,12009,1,1,1,2,50,680,95
6,2014-01-03 17:11:36,2,3,66,462,41898,2454.8588,1482,0,1,...,0,1,12009,1,1,1,2,50,680,95
7,2014-10-29 14:32:19,2,3,66,174,40365,8456.8294,1713,0,0,...,0,2,1571,3,1,1,3,5,89,38
8,2014-12-05 09:58:15,2,3,66,174,21855,8393.1607,1713,0,0,...,0,1,12861,5,1,1,3,5,89,81
9,2013-01-10 18:30:08,37,1,69,923,23649,NaN,2140,1,0,...,2,3,22494,6,1,1,6,204,1776,43


Plan of action:

- Create a good training dataset from the one given:
    - Get rid of click data (remove 0 rows from 'is_booking'
    - Convert date columns to datetime objects - make new columns out of month and year (maybe use to sort?)
    - Create new column for length of trip (srch_co - srch_ci)
    - PCA on destination data
    - Take small smaller training set from whole training file so it is easier to train and test on repeatedly
    - Merge destination file data (after PCA) with training data (use srch_destination_id)
    - Initial EDA of training data
    - Search through dataset for missing values, impute mean when necessary or drop rows with missing values
    - Feature engineering
    - Standardize data?
    - Test/Train split of subset
    - Do some modelling

In [10]:
# attempt to use hdf5 in pandas....aborted - just loading file
# from pandas import HDFStore
# train_hdf = HDFStore('../kag_exp_data/storage.h5')
# train_hdf.put('k_train', data_train, format='table', data_columns=True)
# print train_hdf['k_train'].shape
# data_train.to_hdf('../kag_train', 'data', mode='w', format='table')

Perform PCA on destination data file to still data down to 4 or 5 of it's primary components.

In [11]:
destinations.head(2)

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690


In [12]:
ss = StandardScaler()
destinations_norm = pd.DataFrame(ss.fit_transform(destinations))

pca = PCA(n_components=15)
dest_small = pca.fit_transform(destinations_norm[[i for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["srch_destination_id"] = destinations["srch_destination_id"]
dest_small.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,srch_destination_id
0,-1.197313,1.378518,-0.612534,-0.356261,0.937557,1.340952,2.469440,-0.917367,1.602254,0.771778,-0.091561,0.520935,0.576589,0.615236,-0.376773,0
1,-6.607374,-0.297226,0.706252,-0.832829,-0.249766,0.752353,-0.063990,-0.281699,-0.119281,0.073734,0.267471,-0.363770,0.044325,0.696980,-0.015392,1
2,1.384301,0.505775,1.175688,3.269825,-2.675078,0.257830,-0.056648,-1.720148,1.347416,-2.139900,-1.359655,-2.064712,-0.287617,-0.463452,-0.027959,2
3,-7.354977,-0.437815,0.054926,-0.555849,-0.098661,0.191089,-0.315376,-0.104315,0.464675,0.024518,-0.079485,0.236612,-0.024506,0.767314,0.086682,3
4,-3.733033,-0.627604,0.537632,0.618952,-0.046597,-0.702976,0.792613,-0.005225,-0.088258,0.014265,-0.638072,0.426130,0.782104,0.929578,0.284121,4


In [13]:
# Sum first 5 components (explained variance ratios) from PCA
np.sum(pca.explained_variance_ratio_)

0.72546225686844446

Convert to date features to datetime columns.

In [14]:
data_train['srch_ci'] = pd.to_datetime(data_train.srch_ci)
data_train['srch_co'] = pd.to_datetime(data_train.srch_co)
data_train['date_time'] = pd.to_datetime(data_train.date_time)

In [15]:
data_train['duration'] = data_train.loc[:, 'srch_co'] - data_train.loc[:, 'srch_ci']
data_train['duration'] = data_train.duration.apply(lambda x: x.days)
data_train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,duration
0,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,1,8250,1,1,1,2,50,628,1,4
1,2014-02-27 18:01:32,2,3,66,318,52078,NaN,756,0,1,...,1,8291,1,1,1,2,50,191,2,2
2,2013-06-15 15:38:05,30,4,195,548,56440,NaN,1048,0,1,...,1,1385,1,1,1,0,185,185,58,8
3,2014-11-23 18:02:20,30,4,195,991,47725,NaN,1048,0,0,...,1,8803,1,1,1,3,151,69,36,2
4,2014-01-03 16:30:17,2,3,66,462,41898,2454.8588,1482,0,1,...,2,12009,1,1,1,2,50,680,95,5


In [16]:
data_train[data_train.duration <= 0]

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,duration
590848,2014-11-17 15:55:42,2,3,215,553,11431,NaN,155711,0,0,...,1,8278,1,1,1,2,50,368,18,-9
2052790,2014-11-26 04:14:11,2,3,64,39,35666,NaN,870001,0,0,...,1,20525,1,1,1,3,104,76,46,0
2239487,2014-11-22 04:36:37,53,1,66,196,2428,2317.3493,360392,0,0,...,1,12272,6,1,1,2,50,663,91,-18
2554935,2014-11-25 00:13:38,2,3,66,442,11488,1.6298,772,0,0,...,1,578,1,1,1,2,50,1100,48,-27


In [17]:
# eliminate the four rows that have trip durations that are 0 days or negative days
data_train = data_train[data_train.duration > 0]

Merge small destination dataframe with main training dataset.

In [18]:
data_train.srch_destination_id.unique().shape

(36933,)

In [19]:
dest_small.srch_destination_id.unique().shape

(62106,)

In [20]:
data_dest_train = pd.merge(data_train, dest_small, how='left', on='srch_destination_id')

In [21]:
del data_train
print data_dest_train.shape
print
print data_dest_train.srch_destination_id.unique().shape

(3000689, 40)

(36933,)


There are a million missing values in the 'orig_destination_distance' feature. We are going to use some of the other features to fill in these values. First try KNN, and then a random forest regressor. Could also try gradient boosting or XGBoost if the decision tree works well.

In [2]:
train_df = data_dest_train[data_dest_train.orig_destination_distance.notnull()]
print train_df.shape
train_df.head(2)

NameError: name 'data_dest_train' is not defined

In [23]:
train_df.columns.values

array(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id',
       'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent',
       'hotel_country', 'hotel_market', 'hotel_cluster', 'duration', 0, 1,
       2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype=object)

In [1]:
# try KNN to regress

#knn_n100 = KNeighborsRegressor(n_neighbors=50, weights='uniform')

X = train_df[['site_name', 'posa_continent', 'user_location_country','user_location_region', 
              'user_location_city', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 
              'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 
              'is_booking', 'hotel_continent', 'hotel_country', 'hotel_market', 
              'hotel_cluster', 'duration',0,1,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]

# X = train_df[['user_location_country', 'user_location_region', 'user_location_city', 
#               'hotel_market','hotel_continent', 'hotel_country', 'hotel_cluster',
#               'srch_destination_id']]
y = train_df.orig_destination_distance.values

# ss = StandardScaler()
# Xs = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# knn_n100.fit(X_train, y_train)
# y_pred = knn_n100.predict(X_test)


NameError: name 'train_df' is not defined

In [25]:
# knn_n100.score(X_test, y_test)

In [26]:
def rmsle(pred, true):
    sum_ = 0
    for i in range(len(true)):
        sum_ += (np.log(pred[i]+1) - np.log(true[i]+1))**2
    return np.sqrt(sum_/float(len(true)))

def rmse(pred, true):
    sum_ = 0
    for i in range(len(true)):
        sum_ += (pred[i] - true[i])**2
    return np.sqrt(sum_/float(len(true)))

In [27]:
#rmse(y_pred, y_test)

In [28]:
#plt.scatter(y_pred, y_test)

In [ ]:
#from xgboost import XGBRegressor, XGBClassifier

In [ ]:
rfr = RandomForestRegressor(n_jobs=3)
model = rfr.fit(X_train,y_train)
y_pred = model.predict(X_test)

rmse(y_test, y_pred)

In [ ]:
model.score(X_test, y_test)

In [ ]:
plt.scatter(y_pred, y_test)

The random forest proved much better at predicting the orig_destination_distance, we will use it to replace missing values of orig_destination_distance in the main dataset.

In [ ]:
# xgbr = XGBRegressor()
# model = xgbr.fit(X_train,y_train)
# y_pred_boost = xgbr.predict(X_test)

# rmse(y_test, y_pred_boost)

In [ ]:
# xgbr.score(X_test, y_test)

In [ ]:
# plt.scatter(y_pred_boost, y_test)

In [ ]:
# main dataset (training) with missing values to fill with predicted...
# ...values for orid_destination_distance
train_df_missing = data_dest_train[~data_dest_train.orig_destination_distance.notnull()]

X_miss = train_df_missing[['site_name', 'posa_continent', 'user_location_country','user_location_region', 
              'user_location_city', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 
              'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 
              'is_booking', 'hotel_continent', 'hotel_country', 'hotel_market', 
              'hotel_cluster', 'duration']]

In [ ]:
train_df_missing.shape

In [ ]:
y_missing = rfr.predict(X_miss)

In [ ]:
y_missing

In [ ]:
train_df_missing.loc[:, 'orig_destination_distance'] = y_missing
train_df_missing.head()

In [ ]:
train_data_full = pd.concat([train_df, train_df_missing])
del train_df
del train_df_missing
train_data_full.head().T

In [ ]:
train_data_full.shape

Ok, now we have a dataframe that's been cleaned of missing values, with some of those missing values imputed from other features - above fitting RFR for orig_destination_distance. We also droped about 12000 rows that were missing 'destination' features 0, 1, 2, 3, and 4.

In [ ]:
train_data_full = train_data_full.dropna(axis=1)
train_data_full.reset_index(inplace=True, drop=True)

In [ ]:
train_data_full.isnull().sum()

### Models

In [4]:
y = train_data_full.hotel_cluster.values
drop_cols = ['date_time', 'user_id', 'srch_ci', 'srch_co', 'is_booking', 'hotel_cluster', 'cnt']
keep_cols = [x for x in train_data_full.columns if x not in drop_cols]
keep_cols

NameError: name 'train_data_full' is not defined

In [46]:
y = train_data_full.hotel_cluster.values
drop_cols = ['date_time', 'user_id', 'srch_ci', 'srch_co', 'is_booking', 'hotel_cluster']
keep_cols = [x for x in train_data_full.columns if x not in drop_cols]
X = train_data_full[keep_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

rfc = RandomForestClassifier(n_estimators=8, n_jobs=2)
model = rfc.fit(X_train,y_train)
y_pred = model.predict(X_test)

rmse(y_test, y_pred)

37.340362513070524

In [47]:
sorted(zip(X.columns, model.feature_importances_), key=lambda x: x[1], reverse=True)

[('orig_destination_distance', 0.24248704796058809),
 ('user_location_city', 0.16756069062975587),
 ('user_location_region', 0.098174113528735271),
 ('srch_destination_id', 0.084601723948715749),
 ('channel', 0.071175876848714667),
 ('hotel_market', 0.067364282353167715),
 ('duration', 0.062006217596951366),
 ('srch_adults_cnt', 0.043975567400189493),
 ('srch_children_cnt', 0.02978857879608755),
 ('user_location_country', 0.02375690504521262),
 ('is_mobile', 0.019912701782212109),
 ('site_name', 0.019223696984619999),
 ('srch_rm_cnt', 0.017607027329789679),
 ('srch_destination_type_id', 0.013321537599647699),
 ('hotel_country', 0.013210268773847083),
 ('is_package', 0.0074607658982474576),
 ('hotel_continent', 0.007250900280551178),
 ('posa_continent', 0.0061138825770673619),
 ('cnt', 0.0050082146658990256)]

In [48]:
print y_test[:30]
print y_pred[:30]

[41 13 77 55 49 44  1 21 18 41 91 99 29 83  7 86 98 62 95 24 62 62 22  7 96
 28 19 59 19 13]
[70 13  6  9 49 78  1 21 16  7 48 99 36 83 59 22 98 36 19 45 37 85 38 18 12
 28 18 19 21 94]


In [49]:
prob_arr = model.predict_proba(X_test)

In [50]:
sorted(zip(rfc.classes_, prob_arr[22]), key=lambda x: x[1], reverse=True)[:10]


[(38, 0.25),
 (64, 0.25),
 (36, 0.125),
 (77, 0.125),
 (78, 0.125),
 (15, 0.083333333333333329),
 (6, 0.041666666666666664),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0)]

In [51]:
preds_5 = []
for i in range(len(y_pred)):
    preds_5.append([x[0] for x in sorted(zip(rfc.classes_, 
                                             prob_arr[i]), 
                                         key=lambda x: x[1], 
                                         reverse=True)[:5]])
in_5er = 0
for i in range(len(preds_5)):
    if y_test[i] in preds_5[i]:
        in_5er +=1
print in_5er
print len(preds_5)

361364
900207


In [52]:
in_5er/float(len(preds_5))

0.40142322821306653

In [53]:
import ml_metrics as metrics
target = [[x] for x in y_test.tolist()]
metrics.mapk(target, preds_5, k=5)

0.2600339329361655

In [54]:
del model
del y_pred
del prob_arr
del preds_5
del target

## Run model on contest test data
 - First get test data ready

In [55]:
test_data = pd.read_csv('../kag_exp_data/test.csv')
print test_data.shape
test_data.head()

(2528243, 22)


,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


In [56]:
test_data.isnull().sum()

id                                0
date_time                         0
site_name                         0
posa_continent                    0
user_location_country             0
user_location_region              0
user_location_city                0
orig_destination_distance    847461
user_id                           0
is_mobile                         0
is_package                        0
channel                           0
srch_ci                          21
srch_co                          17
srch_adults_cnt                   0
srch_children_cnt                 0
srch_rm_cnt                       0
srch_destination_id               0
srch_destination_type_id          0
hotel_continent                   0
hotel_country                     0
hotel_market                      0
dtype: int64

Make same modifications to test dataset as we did to the training data. This includes:
    
- Converting date columns to datetime
    - Then calculate duration
    - ~20-30 rows are missing checkin and checkout times - impute most common
- Merge destination_id PCA data to the test data
- Use RFRegressor to fill in missing orig_destination_distance
- Predict hotel cluster with model(s)
- Create submission file

In [57]:
test_data.srch_destination_id.unique().shape

(40718,)

In [58]:
destinations.srch_destination_id.unique().shape

(62106,)

In [59]:
test_data.srch_ci.value_counts()[-30:]

2017-02-12    1
2017-01-06    1
2017-01-30    1
2017-03-28    1
2017-01-03    1
2017-01-09    1
2017-04-15    1
2017-01-04    1
2017-01-05    1
2161-10-00    1
2017-02-26    1
2017-03-04    1
2017-03-05    1
2017-03-07    1
2017-03-03    1
2015-01-04    1
2017-01-17    1
2017-01-15    1
2017-01-12    1
2017-01-11    1
2017-01-18    1
2017-01-19    1
2017-02-02    1
2017-02-03    1
2017-04-09    1
2017-02-04    1
2017-02-06    1
2017-04-26    1
2017-04-25    1
2017-01-10    1
Name: srch_ci, dtype: int64

In [60]:
# one date is really off and is causing problems
test_data[test_data.srch_ci == '2161-10-00']

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
312920,312920,2015-09-27 12:17:51,13,1,46,244,33092,NaN,1041293,0,...,2161-10-00,2016-01-22,2,0,1,6387,1,6,170,1894


In [61]:
test_data.set_value(312920, 'srch_ci', '2016-01-10')
test_data.iloc[312920]

id                                        312920
date_time                    2015-09-27 12:17:51
site_name                                     13
posa_continent                                 1
user_location_country                         46
user_location_region                         244
user_location_city                         33092
orig_destination_distance                    NaN
user_id                                  1041293
is_mobile                                      0
is_package                                     0
channel                                       10
srch_ci                               2016-01-10
srch_co                               2016-01-22
srch_adults_cnt                                2
srch_children_cnt                              0
srch_rm_cnt                                    1
srch_destination_id                         6387
srch_destination_type_id                       1
hotel_continent                                6
hotel_country       

In [62]:
test_data['srch_ci'] = pd.to_datetime(test_data.srch_ci, errors='ignore')
test_data['srch_co'] = pd.to_datetime(test_data.srch_co)
test_data['date_time'] = pd.to_datetime(test_data.date_time)

In [63]:
def fix_duration(x):
    try:
        tmp = x.days
        return tmp
    except AttributeError:
        return 0

In [64]:
test_data['duration'] = test_data.loc[:, 'srch_co'] - test_data.loc[:, 'srch_ci']
test_data['duration'] = test_data.duration.apply(fix_duration)
test_data.head()

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,duration
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-23,2,0,1,12243,6,6,204,27,4
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-15,2,0,1,14474,7,6,204,1540,3
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-27,4,0,1,11353,1,2,50,699,1
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-16,2,0,1,8250,1,2,50,628,2
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-23,2,0,1,11812,1,2,50,538,1


In [65]:
test_data.duration.value_counts(dropna=False)[-25:]

-318    1
-292    1
 86     1
-273    1
 105    1
-365    1
 107    1
 367    1
 136    1
 134    1
 130    1
 129    1
 126    1
 346    1
 123    1
 371    1
 487    1
 118    1
 117    1
 116    1
 373    1
 113    1
 111    1
 110    1
 262    1
Name: duration, dtype: int64

In [66]:
# test_data[test_data.duration <= 0]

In [67]:
data_dest_test = pd.merge(test_data, dest_small, how='left', on='srch_destination_id')
data_dest_test.isnull().sum()

id                                0
date_time                         0
site_name                         0
posa_continent                    0
user_location_country             0
user_location_region              0
user_location_city                0
orig_destination_distance    847461
user_id                           0
is_mobile                         0
is_package                        0
channel                           0
srch_ci                          21
srch_co                          17
srch_adults_cnt                   0
srch_children_cnt                 0
srch_rm_cnt                       0
srch_destination_id               0
srch_destination_type_id          0
hotel_continent                   0
hotel_country                     0
hotel_market                      0
duration                          0
0                             31221
1                             31221
2                             31221
3                             31221
4                           

In [68]:
train_test_df = data_dest_test[data_dest_test.orig_destination_distance.notnull()]
print train_test_df.shape
train_test_df.head(2)

(1680782, 38)


,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,5,6,7,8,9,10,11,12,13,14
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,1.45208,-0.471671,-0.55755,-2.363943,-0.323371,-0.002065,0.679505,-0.755188,-0.469986,-0.510644
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
X = train_test_df[['site_name', 'posa_continent', 'user_location_country','user_location_region', 
                   'user_location_city', 'srch_destination_id', 'srch_destination_type_id', 
                   'hotel_continent', 'hotel_country', 'hotel_market']]

# X = train_test_df[['user_location_country', 'user_location_region', 'user_location_city', 
#                    'hotel_market','hotel_continent', 'hotel_country', 'hotel_cluster',
#                    'srch_destination_id']]
y = train_test_df.orig_destination_distance.values

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

rfr = RandomForestRegressor(n_jobs=3)
rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)

rfr.score(X_test, y_test)


0.99464836636609921

In [71]:
train_test_df_missing = data_dest_test[~data_dest_test.orig_destination_distance.notnull()]

X_miss = train_test_df_missing[['site_name', 'posa_continent', 'user_location_country','user_location_region', 
                                'user_location_city', 'srch_destination_id', 'srch_destination_type_id', 
                                'hotel_continent', 'hotel_country', 'hotel_market']]

y_missing = model.predict(X_miss)


In [72]:
train_test_df_missing.loc[:, 'orig_destination_distance'] = y_missing
train_test_df_missing.head()


,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,5,6,7,8,9,10,11,12,13,14
11,11,2015-08-10 13:35:02,11,3,214,120,44496,4171.766207,56,0,...,-0.420213,3.054430,-0.297359,-1.734010,0.736502,2.151921,-0.416714,0.667463,-0.886885,0.407418
12,12,2015-06-03 10:45:51,2,3,57,342,5021,4751.985590,57,0,...,-3.470022,0.216481,0.549442,-0.097376,1.023986,-0.212570,0.876503,-1.897142,2.137787,0.723465
13,13,2015-06-19 10:16:08,2,3,57,342,5021,5021.238700,57,0,...,1.142044,0.268677,-1.304314,0.513783,-0.006422,1.423886,-1.177643,-0.595946,0.535320,-0.701900
14,14,2015-07-14 08:24:18,2,3,57,342,5021,4458.134647,57,0,...,0.154419,0.810021,0.450141,1.684915,-0.826871,1.456726,-0.541287,0.638163,1.041401,0.879503
15,15,2015-07-14 08:48:48,2,3,57,342,5021,4215.063950,57,0,...,-0.570811,0.861412,-1.180811,1.186131,-0.499144,-0.420039,0.443889,0.785826,2.154080,1.396820


In [73]:
test_data_full = pd.concat([train_test_df, train_test_df_missing])
test_data_full.head().T

,0,1,2,3,4
id,0,1,2,3,4
date_time,2015-09-03 17:09:54,2015-09-24 17:38:35,2015-06-07 15:53:02,2015-09-14 14:49:10,2015-07-17 09:32:04
site_name,2,2,2,2,2
posa_continent,3,3,3,3,3
user_location_country,66,66,66,66,66
user_location_region,174,174,142,258,467
user_location_city,37449,37449,17440,34156,36345
orig_destination_distance,5539.06,5873.29,3975.98,1508.6,66.7913
user_id,1,1,20,28,50
is_mobile,1,1,0,0,0


In [74]:
del train_test_df
del train_test_df_missing

In [75]:
# test_data_full['duration'].fillna(0, inplace=True)
test_data_full.isnull().sum()

id                               0
date_time                        0
site_name                        0
posa_continent                   0
user_location_country            0
user_location_region             0
user_location_city               0
orig_destination_distance        0
user_id                          0
is_mobile                        0
is_package                       0
channel                          0
srch_ci                         21
srch_co                         17
srch_adults_cnt                  0
srch_children_cnt                0
srch_rm_cnt                      0
srch_destination_id              0
srch_destination_type_id         0
hotel_continent                  0
hotel_country                    0
hotel_market                     0
duration                         0
0                            31221
1                            31221
2                            31221
3                            31221
4                            31221
5                   

In [76]:
del destinations
del dest_small
del test_data

In [77]:
# repeat of code from original full training run, used for prediction below
# 'cnt' is removed from the columns because it isn't given in the test set
y = train_data_full.hotel_cluster.values
drop_cols = ['date_time', 'user_id', 'srch_ci', 'srch_co', 'is_booking',
             'cnt', 'hotel_cluster']
keep_cols = [x for x in train_data_full.columns if x not in drop_cols]
X = train_data_full[keep_cols]

rfc = RandomForestClassifier(n_estimators=8, n_jobs=2)
model = rfc.fit(X, y)

In [78]:
# save Ids of current rows
test_full_col = test_data_full.columns.values
test_full_col = test_full_col[:23]             # remove PCA comp. cols
test_ids = test_data_full.id.values
drop_test_cols = ['id', 'date_time', 'user_id', 'srch_ci', 'srch_co', 'cnt']
keep_test_cols = [x for x in test_full_col if x not in drop_test_cols]
Xt = test_data_full[keep_test_cols]
Xt.reset_index(drop=True, inplace=True)

In [79]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# rfc = RandomForestClassifier(n_estimators=10, n_jobs=2)
# model = rfc.fit(X_train,y_train)
y_pred_fin = model.predict(Xt)


In [ ]:
# sorted(zip(X.columns, model.feature_importances_), 
#        key=lambda x: x[1], reverse=True)


In [80]:
prob_arr = model.predict_proba(Xt)

In [95]:
preds_5 = []
for i in range(len(prob_arr)):
    preds_5.append([x[0] for x in sorted(zip(model.classes_, 
                                             prob_arr[i]), 
                                         key=lambda x: x[1], 
                                         reverse=True)[:5]])

In [96]:
preds_5[:10]

[[25, 5, 9, 28, 37],
 [58, 2, 14, 16, 41],
 [6, 28, 31, 39, 42],
 [1, 79, 24, 45, 0],
 [6, 41, 77, 0, 1],
 [32, 42, 48, 77, 0],
 [70, 2, 21, 95, 0],
 [91, 4, 9, 55, 0],
 [79, 1, 41, 45, 0],
 [55, 32, 50, 0, 1]]

In [97]:
import csv

with open("preds_5_1.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(preds_5)

In [98]:
test_ids[-10:]

array([2528170, 2528171, 2528188, 2528199, 2528210, 2528228, 2528229,
       2528230, 2528232, 2528242])

In [99]:
np.savetxt('ids.out', test_ids, delimiter=',')

In [100]:
test_ids.shape

(2528243,)

In [101]:
len(preds_5)

2528243

In [102]:
fin_df = pd.DataFrame(preds_5)
fin_df['id'] = test_ids
fin_df

,0,1,2,3,4,id
0,25,5,9,28,37,0
1,58,2,14,16,41,1
2,6,28,31,39,42,2
3,1,79,24,45,0,3
4,6,41,77,0,1,4
5,32,42,48,77,0,5
6,70,2,21,95,0,6
7,91,4,9,55,0,7
8,79,1,41,45,0,8
9,55,32,50,0,1,9


In [107]:
fin_df.sort_values('id', inplace=True)
fin_df = fin_df[['id', 0, 1, 2, 3, 4]]
fin_dftest

,id,0,1,2,3,4
0,0,25,5,9,28,37
1,1,58,2,14,16,41
2,2,6,28,31,39,42
3,3,1,79,24,45,0
4,4,6,41,77,0,1
5,5,32,42,48,77,0
6,6,70,2,21,95,0
7,7,91,4,9,55,0
8,8,79,1,41,45,0
9,9,55,32,50,0,1


In [104]:
pd.to_pickle(fin_df, 'fin_df')


In [ ]:
# xgb = XGBClassifier(max_depth=3)

# xmodel = xgb.fit(X_train,y_train)
# y_pred = xgb.predict(X_test)
# print rmse(y_test, y_pred)

In [17]:
sub_tmp = pd.read_pickle('./fin_df')
sub_tmp.reset_index(drop=True, inplace=True)

In [18]:
new_col_str = sub_tmp.iloc[:, 1:].applymap(str).apply(lambda x: ' '.join(x), axis=1)
new_col_str.sort_index(inplace=True)
new_col_str[:15]

0      25 5 9 28 37
1     58 2 14 16 41
2     6 28 31 39 42
3      1 79 24 45 0
4       6 41 77 0 1
5     32 42 48 77 0
6      70 2 21 95 0
7       91 4 9 55 0
8      79 1 41 45 0
9      55 32 50 0 1
10     6 33 91 83 0
11    82 6 25 38 48
12     13 0 6 28 51
13     9 58 75 6 17
14    25 4 17 33 72
dtype: object

In [19]:
sub_tmp['hotel_cluster'] = new_col_str
sub_df = sub_tmp[['id', 'hotel_cluster']]
sub_df

,id,pred
0,0,25 5 9 28 37
1,1,58 2 14 16 41
2,2,6 28 31 39 42
3,3,1 79 24 45 0
4,4,6 41 77 0 1
5,5,32 42 48 77 0
6,6,70 2 21 95 0
7,7,91 4 9 55 0
8,8,79 1 41 45 0
9,9,55 32 50 0 1


In [21]:
sub_df.to_csv('sumbission_1.csv', index=False)

In [49]:
# what does the sample submission look like - use to make new one
samp_submis = pd.read_csv('../kag_exp_data/sample_submission.csv')
samp_submis.head()

,id,hotel_cluster
0,0,99 1
1,1,99 1
2,2,99 1
3,3,99 1
4,4,99 1
